In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
from ipywidgets import FloatProgress
from time import sleep
from IPython.display import display
import re
import pickle
from urlextract import URLExtract
import json
import sqlite3

In [29]:
link=[]
products=['womens-clothing-ethnic-wear','womens-western-wear','womens-inner-and-sleepwear','footwear-womens-shoes','womens-watches','women-belts-accessories']
products=['women-belts-accessories']
for i in products:
    c = 114
    while (c <400):
        c=c+1
        result=requests.get(f'https://www.shopclues.com/{i}.html?page={c}')
        g= result.content
        soup = BeautifulSoup(g,"html.parser")
        data=soup.find_all('div',class_='column col4')
        for j in data:
            try:
                link.append("https:"+j.find_all('a')[1].attrs['href'])
            except:
                pass

In [37]:
product_link=list(set(links))

In [38]:
len(product_link)

13197

In [39]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
#Gender
Description=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [40]:
error_link=[]
out_of_stock=[]
for i in product_link:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        s=soup.find('div',class_='prd_mid_info')
        
        if(list(s.find('link',{'itemprop':'availability'}).attrs.values())[1]=='https://schema.org/InStock'):
            
            if(s.find('span',{'id':"sec_list_price_"})!=None):
                MRP.append(s.find('span',{'id':"sec_list_price_"}).text.split('MRP:')[1])
            else:
                if(s.find('span',{'id':'o_price'})!=None):
                    MRP.append(s.find('span',{'id':"o_price"}).text)
                else:
                    MRP.append(s.find('span',{'id':"f_price"}).text)
            Product_Link.append(i)
            Product_Name.append(s.find('h1',{'itemprop':'name'}).text.rstrip().lstrip())
            Product_Brand.append(s.find('span',{'itemprop':'brand'}).text)
            size=s.find('div',{'class':"f_variation"})
            if(size==None):
                Size_Available.append("Not Available")
            else:
                li=size.ul.find_all('span',{'class':'variant'})
                s_l=''
                for j in li:
                    s_l=s_l+j.text+" , "
                Size_Available.append(s_l[:-2])
            Price.append(s.find('span',{'class':'f_price'}).text)
           
                
            
            Description.append(s.find('span',{'itemprop':'description'}).text)
            img=soup.find('div',class_='prd_img_gallery')
            img_list=img.find_all('li',class_="thumb_click")
            if(len(img_list)==1):
                Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                Secondary_Image_Link.append("Not Available")
            else:
                Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                Secondary_Image_Link.append(img_list[1].a.attrs['data-zoom-image'])
        else:
            out_of_stock.append(i)
            
    except:
        error_link.append(i)
        pass

In [ ]:
data=pd.DataFrame({
    "Product":Product_Name,
    "Link":Product_Link,
    "Brand":Product_Brand,
    "Size Available":Size_Available,
    "Selling Price":Price,
    "MRP":MRP,
    "Gender":'Women',
    "Description":Description,
    "Primary_Image":Primary_Image_Link,
    "Secondary_Image":Secondary_Image_Link,
    
})

In [54]:
Product_Link[11884]

'https://www.shopclues.com/kds-paris-fancy-fashion-analog-watch-for-women-134525366.html'

In [55]:
len(error_link)

1290

In [92]:
error_link.index('https://www.shopclues.com/king-creation-blue-net-saree-3.html')

49

In [93]:
e=error_link[:49]+error_link[50:]

In [94]:
len(e)

1289

In [95]:
_Product_Link=[]
_Product_Name=[]
_Product_Brand=[]
_Size_Available=[]
_Price=[]
_MRP=[]
#Gender
_Description=[]
_Primary_Image_Link=[]
_Secondary_Image_Link=[]

In [96]:
#out_of_stock=[]
err=[]
for i in e:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        s=soup.find('div',class_='prd_mid_info')
        
        if(list(s.find('link',{'itemprop':'availability'}).attrs.values())[1]=='https://schema.org/InStock'):
            
            
            if(s.find('span',{'class':'o_price'})!=None):
                _MRP.append(s.find('span',{'class':"o_price"}).text)
            else:
                _MRP.append(s.find('span',{'class':"f_price"}).text)
            _Product_Link.append(i)
            _Product_Name.append(s.find('h1',{'itemprop':'name'}).text.rstrip().lstrip())
            _Product_Brand.append(s.find('span',{'itemprop':'brand'}).text)
            size=s.find('div',{'class':"f_variation"})
            if(size==None):
                _Size_Available.append("Not Available")
            else:
                li=size.ul.find_all('span',{'class':'variant'})
                s_l=''
                for j in li:
                    s_l=s_l+j.text+" , "
                _Size_Available.append(s_l[:-2])
            _Price.append(s.find('span',{'class':'f_price'}).text)
           
                
            
            _Description.append(s.find('span',{'itemprop':'description'}).text)
            img=soup.find('div',class_='prd_img_gallery')
            img_list=img.find_all('li',class_="thumb_click")
            if(img_list==None):
                _Primary_Image_Link.append('Not Available')
                _Secondary_Image_Link.append('Not Available')
            else:
                if(len(img_list)==1):
                    _Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    _Secondary_Image_Link.append("Not Available")
                else:
                    _Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    _Secondary_Image_Link.append(img_list[1].a.attrs['data-zoom-image'])
        else:
            out_of_stock.append(i)
            
    except:
        err.append(i)
        pass

In [97]:
len(err)

30

In [63]:
Product_Link.index('https://www.shopclues.com/king-creation-blue-net-saree-3.html')

431

In [66]:
p=Product_Link[:430]+Product_Link[430:len(Product_Link)-1]

In [70]:
P=Primary_Image_Link[:431]

In [71]:
P.append('Not Available')

In [72]:
P=P+Primary_Image_Link[431:len(Product_Link)-1]

In [76]:
S=Secondary_Image_Link[:431]
S.append('Not Available')
S=S+Secondary_Image_Link[431:len(Product_Link)-1]

In [78]:
data=pd.DataFrame({
    "Product":Product_Name,
    "Link":Product_Link,
    "Brand":Product_Brand,
    "Size Available":Size_Available,
    "Selling Price":Price,
    "MRP":MRP,
    "Gender":'Women',
    "Description":Description,
    "Primary_Image":P,
    "Secondary_Image":S,
    
})

In [80]:
data.to_csv('All_Women_SC.csv',index=False)

In [98]:
data_1=pd.DataFrame({
    "Product":_Product_Name,
    "Link":_Product_Link,
    "Brand":_Product_Brand,
    "Size Available":_Size_Available,
    "Selling Price":_Price,
    "MRP":_MRP,
    "Gender":'Women',
    "Description":_Description,
    "Primary_Image":_Primary_Image_Link,
    "Secondary_Image":_Secondary_Image_Link,
    
})

In [101]:
new_data=data.append(data_1)

In [102]:
new_data.shape

(13144, 10)

In [115]:
new_data.to_csv('All_women_Sc.csv',index=False)

In [116]:
d1=pd.read_csv('All_Men_SC.csv')

In [117]:
d2=pd.read_csv('All_Women_SC.csv')

In [118]:
d=d1.append(d2)

In [119]:
d1.shape

(10072, 10)

In [120]:
d2.shape

(13144, 10)

In [121]:
d.to_csv('All_SC.csv',index=False)

In [122]:
import sqlite3
#from pandas import DataFrame

conn = sqlite3.connect('All_SC.db')
c = conn.cursor()

c.execute('CREATE TABLE products (Product,Link,Brand,Size Available,Selling Price,MRP,Gender,Description,Primary_Image,Secondary_Image)')
conn.commit()

d.to_sql('products', conn, if_exists='replace', index = False)
conn.close

<function Connection.close>

In [2]:
import pandas as pd

In [3]:
data=pd.read_csv('All_SC.csv')

In [4]:
B_men_null=data[:][data['Brand'].isnull()][data['Gender']=='Men']

<ipython-input-4-36ad0b1eeb8e>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  B_men_null=data[:][data['Brand'].isnull()][data['Gender']=='Men']


In [6]:
B_men_null.shape

(3808, 10)

In [9]:
result = requests.get('https://www.shopclues.com/men-belt-black-2.html')
c = result.content
soup = BeautifulSoup(c,"html.parser")
s=soup.find('div',class_='prd_mid_info')

In [10]:
s.find('span',{'itemprop':'brand'}).text==''

True

In [14]:
s.find('span',{'class':'pID'}).a!=None

False

In [15]:
B_men_null['Link'].values

array(['https://www.shopclues.com/evolite-black-stylish-loafers-smart-casuals-for-men-and-boys-137670427.html',
       'https://www.shopclues.com/lorem-analog-black-and-rose-gold-dial-wrist-watch-for-women-lk-221-225-146135815.html',
       'https://www.shopclues.com/men-belt-black-2.html', ...,
       'https://www.shopclues.com/vip-champ-brief-white-brown-pack-of-2-briefs-for-men.html',
       'https://www.shopclues.com/vip-advanta-white-blue-pack-of-2-trunks-for-men-1.html',
       'https://www.shopclues.com/vip-advanta-white-grey-pack-of-2-trunks-for-men.html'],
      dtype=object)

In [16]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
#Gender
Description=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [17]:
error_link=[]
out_of_stock=[]
for i in B_men_null['Link'].values:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        s=soup.find('div',class_='prd_mid_info')
        
        if(list(s.find('link',{'itemprop':'availability'}).attrs.values())[1]=='https://schema.org/InStock'):
            
            if(s.find('span',{'id':"sec_list_price_"})!=None):
                MRP.append(s.find('span',{'id':"sec_list_price_"}).text.split('MRP:')[1])
            else:
                if(s.find('span',{'id':'o_price'})!=None):
                    MRP.append(s.find('span',{'id':"o_price"}).text)
                else:
                    MRP.append(s.find('span',{'id':"f_price"}).text)
            Product_Link.append(i)
            Product_Name.append(s.find('h1',{'itemprop':'name'}).text.rstrip().lstrip())
            if(s.find('span',{'itemprop':'brand'}).text!=''):
                Product_Brand.append(s.find('span',{'itemprop':'brand'}).text)
            else:
                if(s.find('span',{'class':'pID'}).a!=None):
                    Product_Brand.append(s.find('span',{'class':'pID'}).a.text)
                else:
                    Product_Brand.append("Not Available")
                    
            size=s.find('div',{'class':"f_variation"})
            if(size==None):
                Size_Available.append("Not Available")
            else:
                li=size.ul.find_all('span',{'class':'variant'})
                s_l=''
                for j in li:
                    s_l=s_l+j.text+" , "
                Size_Available.append(s_l[:-2])
            Price.append(s.find('span',{'class':'f_price'}).text)
           
                
            
            Description.append(s.find('span',{'itemprop':'description'}).text)
            img=soup.find('div',class_='prd_img_gallery')
            img_list=img.find_all('li',class_="thumb_click")
            if(len(img_list)==1):
                Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                Secondary_Image_Link.append("Not Available")
            else:
                Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                Secondary_Image_Link.append(img_list[1].a.attrs['data-zoom-image'])
        else:
            out_of_stock.append(i)
            
    except:
        error_link.append(i)
        pass

In [28]:
#out_of_stock=[]
err=[]
for i in error_link:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        s=soup.find('div',class_='prd_mid_info')
        
        if(list(s.find('link',{'itemprop':'availability'}).attrs.values())[1]=='https://schema.org/InStock'):
            
            
            if(s.find('span',{'class':'o_price'})!=None):
                MRP.append(s.find('span',{'class':"o_price"}).text)
            else:
                MRP.append(s.find('span',{'class':"f_price"}).text)
            Product_Link.append(i)
            Product_Name.append(s.find('h1',{'itemprop':'name'}).text.rstrip().lstrip())
            if(s.find('span',{'itemprop':'brand'}).text!=''):
                Product_Brand.append(s.find('span',{'itemprop':'brand'}).text)
            else:
                if(s.find('span',{'class':'pID'}).a!=None):
                    Product_Brand.append(s.find('span',{'class':'pID'}).a.text)
                else:
                    Product_Brand.append("Not Available")
            size=s.find('div',{'class':"f_variation"})
            if(size==None):
                Size_Available.append("Not Available")
            else:
                li=size.ul.find_all('span',{'class':'variant'})
                s_l=''
                for j in li:
                    s_l=s_l+j.text+" , "
                Size_Available.append(s_l[:-2])
            Price.append(s.find('span',{'class':'f_price'}).text)
           
                
            
            Description.append(s.find('span',{'itemprop':'description'}).text)
            img=soup.find('div',class_='prd_img_gallery')
            img_list=img.find_all('li',class_="thumb_click")
            if(img_list==None):
                Primary_Image_Link.append('Not Available')
                Secondary_Image_Link.append('Not Available')
            else:
                if(len(img_list)==1):
                    Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    Secondary_Image_Link.append("Not Available")
                else:
                    Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    Secondary_Image_Link.append(img_list[1].a.attrs['data-zoom-image'])
        else:
            out_of_stock.append(i)
            
    except:
        err.append(i)
        pass

In [29]:
data1=pd.DataFrame({
    "Product":Product_Name,
    "Link":Product_Link,
    "Brand":Product_Brand,
    "Size Available":Size_Available,
    "Selling Price":Price,
    "MRP":MRP,
    "Gender":'Men',
    "Description":Description,
    "Primary_Image":Primary_Image_Link,
    "Secondary_Image":Secondary_Image_Link,
    
})

In [30]:
data1.shape

(3755, 10)

In [32]:
data1.to_csv('Men_Null.csv',index=False)

In [22]:
W_null=data[:][data['Brand'].isnull()][data['Gender']=='Women']

<ipython-input-22-754e7deed7e5>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  W_null=data[:][data['Brand'].isnull()][data['Gender']=='Women']


In [23]:
W_null.shape

(5316, 10)

In [33]:
error_link=[]
out_of_stock=[]
for i in W_null['Link'].values:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        s=soup.find('div',class_='prd_mid_info')
        
        if(list(s.find('link',{'itemprop':'availability'}).attrs.values())[1]=='https://schema.org/InStock'):
            
            if(s.find('span',{'id':"sec_list_price_"})!=None):
                MRP.append(s.find('span',{'id':"sec_list_price_"}).text.split('MRP:')[1])
            else:
                if(s.find('span',{'id':'o_price'})!=None):
                    MRP.append(s.find('span',{'id':"o_price"}).text)
                else:
                    MRP.append(s.find('span',{'id':"f_price"}).text)
            Product_Link.append(i)
            Product_Name.append(s.find('h1',{'itemprop':'name'}).text.rstrip().lstrip())
            if(s.find('span',{'itemprop':'brand'}).text!=''):
                Product_Brand.append(s.find('span',{'itemprop':'brand'}).text)
            else:
                if(s.find('span',{'class':'pID'}).a!=None):
                    Product_Brand.append(s.find('span',{'class':'pID'}).a.text)
                else:
                    Product_Brand.append("Not Available")
                    
            size=s.find('div',{'class':"f_variation"})
            if(size==None):
                Size_Available.append("Not Available")
            else:
                li=size.ul.find_all('span',{'class':'variant'})
                s_l=''
                for j in li:
                    s_l=s_l+j.text+" , "
                Size_Available.append(s_l[:-2])
            Price.append(s.find('span',{'class':'f_price'}).text)
           
                
            
            Description.append(s.find('span',{'itemprop':'description'}).text)
            img=soup.find('div',class_='prd_img_gallery')
            img_list=img.find_all('li',class_="thumb_click")
            if(img_list==None):
                Primary_Image_Link.append('Not Available')
                Secondary_Image_Link.append('Not Available')
            else:
                if(len(img_list)==1):
                    Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    Secondary_Image_Link.append("Not Available")
                else:
                    Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    Secondary_Image_Link.append(img_list[1].a.attrs['data-zoom-image'])
        else:
            out_of_stock.append(i)
            
    except:
        error_link.append(i)
        pass

In [34]:
len(error_link)

883

In [44]:
len(Secondary_Image_Link)

8178

In [46]:
Product_Link.index('https://www.shopclues.com/king-creation-blue-net-saree-3.html')

3924

In [47]:
S=Secondary_Image_Link[:3924]
S.append('Not Available')
S=S+Secondary_Image_Link[3924:len(Product_Link)-1]

In [48]:
P=Primary_Image_Link[:3924]
P.append('Not Available')
P=P+Primary_Image_Link[3924:len(Product_Link)-1]

In [50]:
Primary_Image_Link=P
Secondary_Image_Link=S

In [51]:
data_3=pd.DataFrame({
    "Product":Product_Name,
    "Link":Product_Link,
    "Brand":Product_Brand,
    "Size Available":Size_Available,
    "Selling Price":Price,
    "MRP":MRP,
    "Gender":'Women',
    "Description":Description,
    "Primary_Image":Primary_Image_Link,
    "Secondary_Image":Secondary_Image_Link,
    
})

In [52]:
data_3.to_csv('All_n.csv',index=False)

In [53]:
error_link.index('https://www.shopclues.com/king-creation-blue-net-saree-3.html')

0

In [54]:
#out_of_stock=[]
err=[]
for i in error_link[1:]:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        s=soup.find('div',class_='prd_mid_info')
        
        if(list(s.find('link',{'itemprop':'availability'}).attrs.values())[1]=='https://schema.org/InStock'):
            
            
            if(s.find('span',{'class':'o_price'})!=None):
                MRP.append(s.find('span',{'class':"o_price"}).text)
            else:
                MRP.append(s.find('span',{'class':"f_price"}).text)
            Product_Link.append(i)
            Product_Name.append(s.find('h1',{'itemprop':'name'}).text.rstrip().lstrip())
            if(s.find('span',{'itemprop':'brand'}).text!=''):
                Product_Brand.append(s.find('span',{'itemprop':'brand'}).text)
            else:
                if(s.find('span',{'class':'pID'}).a!=None):
                    Product_Brand.append(s.find('span',{'class':'pID'}).a.text)
                else:
                    Product_Brand.append("Not Available")
            size=s.find('div',{'class':"f_variation"})
            if(size==None):
                Size_Available.append("Not Available")
            else:
                li=size.ul.find_all('span',{'class':'variant'})
                s_l=''
                for j in li:
                    s_l=s_l+j.text+" , "
                Size_Available.append(s_l[:-2])
            Price.append(s.find('span',{'class':'f_price'}).text)
           
                
            
            Description.append(s.find('span',{'itemprop':'description'}).text)
            img=soup.find('div',class_='prd_img_gallery')
            img_list=img.find_all('li',class_="thumb_click")
            if(img_list==None):
                Primary_Image_Link.append('Not Available')
                Secondary_Image_Link.append('Not Available')
            else:
                if(len(img_list)==1):
                    Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    Secondary_Image_Link.append("Not Available")
                else:
                    Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    Secondary_Image_Link.append(img_list[1].a.attrs['data-zoom-image'])
        else:
            out_of_stock.append(i)
            
    except:
        err.append(i)
        pass

In [55]:
err

['https://www.shopclues.com/bra-push-pads-3-pairs-high-quality-material.html']

In [56]:
len(Product_Link)

9060

In [57]:
data_n=pd.DataFrame({
    "Product":Product_Name,
    "Link":Product_Link,
    "Brand":Product_Brand,
    "Size Available":Size_Available,
    "Selling Price":Price,
    "MRP":MRP,
    "Gender":'Women',
    "Description":Description,
    "Primary_Image":Primary_Image_Link,
    "Secondary_Image":Secondary_Image_Link,
    
})

In [168]:
data_n.to_csv("All_Null_SC.csv",index=False)

In [172]:
data_n.shape

(9060, 10)

In [173]:
d1=pd.read_csv('Men_Null.csv')

In [174]:
d1.isnull().sum()

Product            0
Link               0
Brand              5
Size Available     0
Selling Price      0
MRP                0
Gender             0
Description        0
Primary_Image      1
Secondary_Image    0
dtype: int64

In [169]:
d2=pd.read_csv("All_Null_SC.csv")

In [170]:
d2.isnull().sum()

Product             0
Link                0
Brand              19
Size Available      0
Selling Price       0
MRP                 0
Gender              0
Description         0
Primary_Image      10
Secondary_Image     4
dtype: int64

In [133]:
d2=d2.drop(columns='Gender',axis=1)

In [134]:
d1=d1.drop(columns='Gender',axis=1)

In [135]:
d1.Link.values

array(['https://www.shopclues.com/evolite-black-stylish-loafers-smart-casuals-for-men-and-boys-137670427.html',
       'https://www.shopclues.com/lorem-analog-black-and-rose-gold-dial-wrist-watch-for-women-lk-221-225-146135815.html',
       'https://www.shopclues.com/men-belt-black-2.html', ...,
       'https://www.shopclues.com/vip-champ-brief-white-brown-pack-of-2-briefs-for-men.html',
       'https://www.shopclues.com/vip-advanta-white-blue-pack-of-2-trunks-for-men-1.html',
       'https://www.shopclues.com/vip-advanta-white-grey-pack-of-2-trunks-for-men.html'],
      dtype=object)

In [125]:
d2[:][d2.Link=='https://www.shopclues.com/vip-champ-brief-white-brown-pack-of-2-briefs-for-men.html']

,Product,Link,Brand,Size Available,Selling Price,MRP,Description,Primary_Image,Secondary_Image
3752,"VIP Champ Brief White,Brown Pack of 2 Briefs f...",https://www.shopclues.com/vip-champ-brief-whit...,VIP,"110 , 100 , 80",₹270,₹304,"Buy VIP Champ Brief White,Brown Pack of 2 Brie...",https://cdn.shopclues.com/images/detailed/4825...,https://cdn.shopclues.com/images/detailed/4826...


In [137]:
for i  in d1.Link.values:
    index_names = d2[ d2['Link'] == i ].index
    d2.drop(index_names, inplace = True)

In [139]:
d2['Gender']='Women'

In [140]:
d1['Gender']='Men'

In [162]:
d1.isnull().sum()

Product            0
Link               0
Brand              5
Size Available     0
Selling Price      0
MRP                0
Description        0
Primary_Image      1
Secondary_Image    0
Gender             0
dtype: int64

In [141]:
d=d1.append(d2)

In [161]:
d.isnull().sum()

Product             0
Link                0
Brand              19
Size Available      0
Selling Price       0
MRP                 0
Description         0
Primary_Image      10
Secondary_Image     4
Gender              0
dtype: int64

In [143]:
d.shape

(9051, 10)

In [103]:
n=pd.read_csv("All_Null_SC.csv")

In [104]:
n.shape

(9060, 10)

In [158]:
data=pd.read_csv('All_SC.csv')

In [159]:
data.shape

(23216, 10)

In [149]:
data=data.dropna()

In [150]:
data.shape

(14063, 10)

In [151]:
data=data.append(d)

In [153]:
data.to_csv("All_Shopclues.csv",index=False)

In [154]:
dd=pd.read_csv('All_Shopclues.csv')

In [156]:
dd[:][dd['Gender']=='Women'].shape

(13102, 10)

In [160]:
dd.shape

(23114, 10)

In [71]:
import sqlite3
#from pandas import DataFrame

conn = sqlite3.connect('All_ShopClues.db')
c = conn.cursor()

c.execute('CREATE TABLE products (Product,Link,Brand,Size Available,Selling Price,MRP,Gender,Description,Primary_Image,Secondary_Image)')
conn.commit()

data.to_sql('products', conn, if_exists='replace', index = False)
conn.close

C:\Users\alokt\anaconda3\lib\site-packages\pandas\core\generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


<function Connection.close>

In [72]:
data.isnull().sum()

Product            0
Link               0
Brand              0
Size Available     0
Selling Price      0
MRP                0
Gender             0
Description        0
Primary_Image      0
Secondary_Image    0
dtype: int64